## e011 EDA
 * startified_k_fold の妥当性調査

In [1]:
import pandas as pd
import numpy as np
import gc
import pickle
import pyarrow.parquet as pq
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

import scipy
from scipy import signal
from scipy.signal import butter

from matplotlib import pyplot as plt
import seaborn as sns

import sys
sys.path.append('../../guchio_utils/')

In [2]:
#plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
# %config InlineBackend.figure_formats = {'png', 'retina'}
%load_ext autoreload
%autoreload 1

### OOF analysis

In [2]:
#trn_df = pd.read_parquet('../inputs/origin/train.parquet.zip', compression='zip')
trn_df = pq.read_pandas('../inputs/origin/train.parquet').to_pandas()
trn_meta_df = pd.read_csv('../inputs/origin/metadata_train.csv')

In [3]:
with open('../oofs/e011_2019-01-17-17-42-05_0.5739_0.311_oofs.pkl', 'rb') as fin:
    oofs = pickle.load(fin)

In [5]:
oofs[0]

[array([   0,    1,    2, ..., 1746, 1747, 1748]),
 array([1575, 1576, 1577, ..., 3487, 3488, 3489]),
 array([3372, 3373, 3374, ..., 5267, 5268, 5269]),
 array([4768, 4769, 4815, ..., 6993, 6994, 6995]),
 array([6654, 6655, 6656, ..., 8709, 8710, 8711])]

In [10]:
for labels in oofs[0]:
#    print(trn_meta_df.set_index('signal_id').loc[labels].target.sum())
    print(trn_meta_df.set_index('signal_id').loc[labels].id_measurement.nunique() * 3, trn_meta_df.set_index('signal_id').loc[labels].reset_index().signal_id.nunique())

1743 1743
1746 1743
1752 1742
1749 1742
1743 1742
